# Imports

In [44]:
import os
import time
import datetime
import warnings
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F

Check for cuda

In [2]:
torch.cuda.is_available()

False

Setting seeds

In [45]:
def set_seed(seed: int = 4) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

## Creating dataset

In [12]:
weight = 0.7
bias = 0.3

start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

In [50]:
len(X), len(y)

(50, 50)

In [13]:
X[:5], y[:5]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560]]))

## Plots

In [49]:
def plot_dataset(train_data,
                 train_label,
                 test_data,
                 test_label,
                 valid_data = None,
                 valid_label = None,
                 fig_size = (10, 7),
                 predictions=None) -> None:
    plt.figure(figsize=fig_size)

    plt.scatter(train_data,
                train_label,
                c="g",
                label = "Training Data")
    
    plt.scatter(test_data,
                test_label,
                c="o",
                label = "Testing Data")
    
    if valid_data is None or valid_label is None:
        plt.scatter(valid_data,
                    valid_label,
                    c="b",
                    label = "Validation Data")
    
    plt.legend()

## Creating Linear Regression Model

In [14]:
class LinearRegressionModel(nn.Module):  # Wea re creating a subclass from  nn.module
    def __init__(self):
        super().__init__()

        # Initializing the model parameters: Weights and Bias
        '''
        requires_grad=True; means PyTorch will track the gradients of the specific params for use with torch.autograd

        '''
        self.weights = nn.Parameter(torch.randn(1,
                                                requires_grad=True,
                                                dtype=torch.float))
        
        self.bias = nn.Parameter(torch.randn(1, 
                                             requires_grad=True,
                                             dtype = torch.float))
        
    # Forward Pass, Here we are overwriting the forward method, given by default from nn.module
    # def forward(self,
    #             x: torch.Tensor) -> torch.Tensor:
    #     return self.weights * x + self.bias  # The Linear regression Formula
    
    def forward_lin_reg(self,
                        x: torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias


In [46]:
set_seed()
lin_reg_0 = LinearRegressionModel()

Random seed set as 4


In [47]:
lin_reg_0

LinearRegressionModel()

In [48]:
list(lin_reg_0.parameters())

[Parameter containing:
 tensor([-1.6053], requires_grad=True),
 Parameter containing:
 tensor([0.2325], requires_grad=True)]